In [1]:
import os
import pandas as pd

# Extract all csv files from the 'RAMCore' directory again
ram_core_dir = r"../datasources/RAM/RAMCore"
    
ram_core_files = [os.path.join(ram_core_dir, file) for file in os.listdir(ram_core_dir) if file.endswith('.csv')]

sbt_data = {}

for file_path in ram_core_files:
    df = pd.read_csv(file_path)
    if 'SBT' in df.columns:  # Check if the DataFrame contains data for the Southern Bluefin Tuna
        metric = os.path.basename(file_path)  # Get the name of the metric from the file name
        sbt_data[metric] = df[['Unnamed: 0', 'SBT']]  # Include the 'Year' column

In [2]:
sbt_combined = pd.DataFrame()

for name, df in sbt_data.items():
    if len(df) == 71:
        df = df.drop(df.index[-1])  # Drop the last row
    df.set_index('Unnamed: 0', inplace=True)  # Set 'Unnamed: 0' as the index
    df.index.rename('Year', inplace=True)
    if sbt_combined.empty:
        sbt_combined = df.rename(columns={'SBT': name.split('.')[0]})
    else:
        sbt_combined[name.split('.')[0]] = df['SBT']

sbt_combined = sbt_combined.fillna(0)

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1('BlueBalance', className='text-center my-4'),
        ], width=12),
    ]),
    dbc.Row([
        dbc.Col([
            html.Label('Metrics'),
            dcc.Dropdown(
                id='metric-dropdown',
                options=[{'label': i, 'value': i} for i in sbt_combined.columns],
                value='Bio',
                style={
                    'color': 'black'
                }
            ),
        ], width=6),
        dbc.Col([
            html.Label('Years'),
            dcc.RangeSlider(
                id='year-slider',
                min=sbt_combined.index.min(),
                max=sbt_combined.index.max(),
                value=[sbt_combined.index.min(), sbt_combined.index.max()],
                marks={str(year): str(year) for year in range(sbt_combined.index.min(), sbt_combined.index.max() + 1, 10)},
                step=None
            ),
        ], width=6),
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='time-series-chart'),
        ], width=9),
        dbc.Col([
            html.Label('Catch Rate'),
            dcc.Slider(
                id='catch-rate-slider',
                min=0,
                max=400,
                value=100,
                marks={i: '{} %'.format(i) for i in range(0, 401, 50)},
                step=None
            ),
            html.Label('Water Temp'),
            dcc.Slider(
                id='water-temp-slider',
                min=-10,
                max=10,
                value=0,
                marks={i: '{} °C'.format(i) for i in range(-10, 11, 2)},
                step=None
            ),
        ], width=3),
    ]),
    dbc.Row([
        dbc.Col([
            html.H3('Definitions:'),
                html.P('Bio: Biomass. Total weight of all individuals in the stock.'),
                html.P('Btrend: Trend in biomass over time.'),
                html.P('BvB: Biomass relative to the biomass reference point.'),
                html.P('BvT: Biomass relative to a target biomass.'),
                html.P('CPUE: Catch Per Unit Effort. The number of fish caught per unit of fishing effort.'),
                html.P('CTrend: Trend in catch over time.'),
                html.P('CvMEANC: Catch relative to the mean catch.'),
                html.P('CvMSY: Catch relative to the Maximum Sustainable Yield.'),
                html.P('FF: Fishing Factor. A dimensionless factor that reflects fishing pressure.'),
                html.P('MCatch: Maximum sustainable catch. The highest catch that can be continuously taken from a stock under existing environmental conditions.'),
                html.P('R: Recruitment. The number of fish that have grown or survived to become part of the exploitable stock.'),
                html.P('SSB: Spawning Stock Biomass. Total weight of all sexually mature individuals in the stock.'),
                html.P('Survey: A measure from a scientific fish survey, often related to abundance or biomass.'),
                html.P('U: Exploitation rate. The proportion of the fish stock removed by fishing.'),
                html.P('Utrend: Trend in exploitation rate over time.'),
                html.P('UvT: Exploitation rate relative to a target exploitation rate.'),
                html.P('UvU: Exploitation rate relative to the exploitation rate at MSY.')
        ]),
    ]),
], fluid=True)

@app.callback(
    Output('time-series-chart', 'figure'),  
    [Input('metric-dropdown', 'value'),
     Input('year-slider', 'value'),
     Input('water-temp-slider', 'value'),
     Input('catch-rate-slider', 'value')]
)
def update_graph(selected_metric, selected_years, water_temp, catch_rate):
    filtered_df = sbt_combined[(sbt_combined.index >= selected_years[0]) & (sbt_combined.index <= selected_years[1])].copy()

    if selected_metric == 'Bio':
        # Apply the algorithm to adjust the population size
        filtered_df['Bio'] *= (1 + 0.01 * -water_temp)  # Adjust for water temperature
        filtered_df['Bio'] *= (1 - 0.01 * (catch_rate - 100))  # Adjust for fishing rate

        # Fill NaN values with 0
        filtered_df['Bio'].fillna(0, inplace=True)

    # Add the geographical coordinates
    filtered_df['Latitude'] = -21
    filtered_df['Longitude'] = -20

    # Create a scatter plot on a world map
    fig = px.scatter_geo(filtered_df, lat='Latitude', lon='Longitude', 
                         size=selected_metric, animation_frame=filtered_df.index,
                         title=f'Southern Bluefin Tuna {selected_metric} Over Time',
                         projection='natural earth')

    return fig

app.run_server(debug=False)

<ipython-input-3-8566ec8d6db6>:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-3-8566ec8d6db6>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jul/2023 15:36:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 15:36:32] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 15:36:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 15:36:32] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 15:36:32] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 15:36:32] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 15:36:32] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 15:36:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 15:36:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 15:37:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 15:37:24] "POST /_dash-